In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

img_width, img_height = 150, 150
train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
epochs = 25
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#Creating the model
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [12]:
train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                             target_size=(img_width,img_height),
                                                             class_mode='binary',
                                                             batch_size=batch_size)

train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                   target_size=(img_width,img_height),
                                                   class_mode='binary',
                                                   batch_size=batch_size)


hist = model.fit_generator(train_generator,
                    steps_per_epoch=1000 // batch_size,
                    epochs=2,
                    validation_data=validation_generator,
                    validation_steps=500 // batch_size)

Found 1000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/2
62/62 [==============================] - 74s 1s/step - loss: 0.5908 - acc: 0.7117 - val_loss: 0.5591 - val_acc: 0.7010
Epoch 2/2
62/62 [==============================] - 68s 1s/step - loss: 0.5959 - acc: 0.6835 - val_loss: 0.5647 - val_acc: 0.7030


In [33]:
test_dir = 'data/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width,img_height),
        shuffle = True,
        class_mode='binary',
        batch_size=1)

filenames = test_generator.filenames

nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)
print(predict.shape)
print(len(predict), end="\n")
for v in predict:
    print(v, end="\n")

Found 17 images belonging to 2 classes.
(17, 1)
17
[0.23568447]
[0.08779854]
[0.43195555]
[0.40016642]
[0.44954678]
[0.5967686]
[0.87198335]
[0.99995005]
[0.7811682]
[0.5122932]
[0.20184113]
[0.37273067]
[0.00517031]
[0.28269246]
[0.42123234]
[0.4878804]
[0.39350954]


In [8]:
print(hist.history.keys())

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [9]:
print(hist)

In [13]:
print(len(hist.history))

4
